In [63]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [64]:
import pandas as pd
import requests
import json
import pprint


# Testing HL Api for PULL transfer

---

## Consts

In [80]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
context_broker = "http://fiware-orion:1026"
streaming_processor = "http://streaming-testing-service:1237"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [81]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '4423ec2a-908f-4fc7-8a56-2c246462c887',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '4423ec2a-908f-4fc7-8a56-2c246462c887',
 'dct:issued': '2024-11-26T10:19:55.413449',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [82]:
payload = {
    "dcat:endpointURL": context_broker + "/v2/entities"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'b74619d1-ade3-4b38-94ac-9bf28b86e82a',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '',
 'dcat:endpointURL': 'http://fiware-orion:1026/v2/entities',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'b74619d1-ade3-4b38-94ac-9bf28b86e82a',
 'dct:issued': '2024-11-26T10:20:08.861504',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [83]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': '261366f3-8551-4b71-8f7b-a5ef68c92386',
 'data_service_id': 'b74619d1-ade3-4b38-94ac-9bf28b86e82a',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [84]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://ds-consumer:1235/87ef55b0-b061-4bef-98f3-d3dba91d8149',
 'callbackId': '87ef55b0-b061-4bef-98f3-d3dba91d8149',
 'consumerPid': 'urn:uuid:877d646a-7e91-40b6-b3b4-a86c125fd223'}


## Request transfer

In [85]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
  "agreementId": agreement_pid,
  "format": "fiware+pull",
  "dataAddress": None,
  "callbackAddress": callbackAddress,
  "callbackId": callbackId,
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:877d646a-7e91-40b6-b3b4-a86c125fd223',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:877d646a-7e91-40b6-b3b4-a86c125fd223',
                     'dspace:providerPid': 'urn:uuid:ba337be6-ff83-4ce3-a579-b7ea11eb9867',
                     'dspace:state': 'dspace:REQUESTED'}}


## Get data address

In [86]:
import requests

consumerPidUuid = consumerPid.replace("urn:uuid:", "")
url = data_space_consumer + "/api/v1/data-address/" + consumerPidUuid

payload = ""
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
dataPlaneAddress = response.json()["dataPlaneAddress"]

pprint.pprint(response.json())

{'dataPlaneAddress': 'http://ds-consumer:1235/87ef55b0-b061-4bef-98f3-d3dba91d8149/data/pull/877d646a-7e91-40b6-b3b4-a86c125fd223'}


## Use data address to get data from data space

In [92]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df["CO"]

0    {'type': 'Number', 'value': '16', 'metadata': {}}
Name: CO, dtype: object

## Suspend transfer

In [93]:
url = data_space_consumer + "/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:877d646a-7e91-40b6-b3b4-a86c125fd223',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:877d646a-7e91-40b6-b3b4-a86c125fd223',
                     'dspace:providerPid': 'urn:uuid:ba337be6-ff83-4ce3-a579-b7ea11eb9867',
                     'dspace:state': 'dspace:SUSPENDED'}}


## Test data access has been suspended

**Disclaimer**: Data shouldn't appear. But for this deployment does appear because an internal refactor we're facing.
In a couple of days this functionality will work like is expected.

In [74]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

,id,type,CO,relativeHumidity,temperature
0,AirQualityUnit01,AirQualityUnit,"{'type': 'Number', 'value': '82', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}"


## Restart transfer

In [75]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:aee5d09f-fad5-45dd-9907-586d41b02ee3',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:aee5d09f-fad5-45dd-9907-586d41b02ee3',
                     'dspace:providerPid': 'urn:uuid:0bb06df8-6569-45b7-9e3b-be9ff3c79502',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

In [76]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

,id,type,CO,relativeHumidity,temperature
0,AirQualityUnit01,AirQualityUnit,"{'type': 'Number', 'value': '82', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}"


## Complete transfer

In [77]:
url = data_space_consumer + "/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:aee5d09f-fad5-45dd-9907-586d41b02ee3',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:aee5d09f-fad5-45dd-9907-586d41b02ee3',
                     'dspace:providerPid': 'urn:uuid:0bb06df8-6569-45b7-9e3b-be9ff3c79502',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

In [78]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

,id,type,CO,relativeHumidity,temperature
0,AirQualityUnit01,AirQualityUnit,"{'type': 'Number', 'value': '8', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}"


## Protocol state cannot change

In [79]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:aee5d09f-fad5-45dd-9907-586d41b02ee3',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
